In [ ]:
import pandas as pd
import numpy as np
df = pd.read_json('data\data_bike_exchange.json', lines=True)
df.head(10)

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.reset_option('all')

In [ ]:
df[['Type', 'Riding Style']][df['Type'].str.contains('Road')].head(10)

In [ ]:
def print_nans_perc(df):
    n = len(df)
    nans_perc = ((df.isna().sum() / n) * 100).sort_values(ascending=True)
    print('len - ', n)
    for index, val in zip(nans_perc.index, nans_perc):
        print(index + ': ', val)
print_nans_perc(df)

In [ ]:
bike_types = df['Type'].unique()
for Btype in bike_types:
    print('\n\n'+ Btype + ':\n')
    print_nans_perc(df[df['Type'] == Btype])

In [ ]:
usefull_cols = ['ID','Title','Type','Price now','Brand','Item condition', 'Material', 'Rear Derailleur', 'Groupset', 'Braking Type']
usefull_cols_road = ['ID','Title','Price now','Brand','Item condition', 'Material', 'Groupset', 'Braking Type']

In [ ]:
roadies_orig = df[df['Type'] == 'Road Bikes'][usefull_cols_road]
print_nans_perc(roadies_orig)

In [ ]:
roadies = roadies_orig.copy()
roadies.rename(columns={'Price now': 'Price'}, inplace=True)
roadies['Price'].replace('[A-Za-z, \$]','',inplace=True, regex=True)
roadies = roadies[roadies['Price'] != '']
roadies['Price'] = roadies['Price'].astype(float)
roadies

In [ ]:
roadies.rename(columns={'Item condition': 'isNew'}, inplace=True)
roadies['isNew'].replace({'Used': 0, 'New': 1},inplace=True, regex=True)
roadies['isNew'] = roadies['isNew'].astype(int)
roadies

In [ ]:
material_dict = {'Chromoly': 0.15, 'Steel': 0.3, 'Aluminum': 0.45, 'Titanium': 0.6, 'Carbon Alloy Composite': 0.75, 'Carbon': 0.9}
roadies['material_num'] = roadies['Material'].replace(material_dict, regex=True)
roadies['material_num'] = roadies['material_num'].astype(float)
roadies

In [ ]:
roadies.groupby('Groupset').size()

In [ ]:
roadies = roadies[roadies['Groupset'] != 'N/A']
roadies['e_shift_num'] = roadies['Groupset'].apply(lambda x: 1 if any(ele in x for ele in ['Di2', 'AXS', 'eTap']) else 0)

gropset_dict = {'Campagnolo Veloce': 0.7,'Campagnolo Centaur': 0.8, 'Campagnolo Chorus': 0.9, 'Campagnolo Super Record': 1,
       'SRAM Apex': 0.6, 'SRAM Rival': 0.7, 'SRAM Force': 0.8, 'SRAM Red': 0.9,  
       'SRAM Force AXS': 0.85, 'SRAM Red AXS': 1, 'SRAM Red eTap': 1,
       'Shimano Tourney': 0.3, 'Shimano Claris': 0.4, 'Shimano Sora': 0.5,'Shimano Tiagra': 0.6, 'Shimano 105': 0.7,'Shimano Ultegra': 0.8, 'Shimano Dura Ace': 0.9,
       'Shimano XTR Di2': 1, 'Shimano Ultegra Di2': 0.85 ,'Shimano Dura Ace Di2': 1, 
       'Shimano A070': 0.3, 'Shimano GRX': 0.7, 'Shimano R2300': 0.3 , 'Shimano R460': 0.3,
       }
roadies['groupset_num'] = roadies['Groupset'].replace(gropset_dict, regex=True, inplace=False)
roadies

In [ ]:
roadies.rename(columns={'Braking Type': 'brake_num'}, inplace=True)
roadies['brake_num'].replace({'Disc Brakes': 1, 'Rim Brakes': 0.5}, inplace=True, regex=True)
roadies

In [ ]:
import re

def find_year(x):
    four_num = re.findall(r'[0-9]{4}', x)
    four_num = [int(num) for num in four_num]
    for num in four_num:
        if 1990 <= num <= 2024:
            return num
    return np.nan

roadies['Production_year'] = roadies['Title'].apply(find_year)
roadies['Production_year'].fillna(roadies['Production_year'].median(), inplace=True)
roadies['Production_year']

In [ ]:
num_cols = ['ID', 'Price','Production_year', 'isNew', 'material_num', 'groupset_num', 'e_shift_num', 'brake_num']

In [ ]:
roadies_num = roadies[num_cols]
roadies_num.to_json('data\exch_road.json', lines=True, orient='records')
roadies_num